In [21]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'
import pandas as pd
import numpy as np
import sys
from io import StringIO
import glob
import json
if not hasattr(sys, "version_info") or sys.version_info < (3, 5):
  raise SystemExit("This program requires Python 3.5 or later.")

import matplotlib
#matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
data_sources = glob.glob("data/*_shops.xml")

import tabletext
# tags
timetag = "lastupdatetime"
shopTag = "shop"


In [22]:
def processShop(tree, time):
    ret = []
    for good in tree:
        goodData = {}
        goodData["ttid"] = int(good.findall("ttid")[0].text)
        goodData["time"] = time
        goodData["shop"] = tree.attrib["name"]
        goodData["shopid"] = tree.attrib["id"]
        goodData["shopmoney"] = tree.attrib["money"]
        sellElems = good.findall("sell")
        if sellElems:
            goodData["sell"] = float(sellElems[0].text)
        else:
            goodData["sell"] = 0.0
            
        buyElems = good.findall("buy")
        if buyElems:
            goodData["buy"] = float(buyElems[0].text)
        else:
            goodData["buy"] = 0.0
        goodData["count"] = int(good.findall("count")[0].text)
        goodData["thingname"] = good.findall("thingname")[0].text
        ret.append(goodData)
    return ret

def selectTtid(ttid, data):
    ret = []
    for row in data:
        if row["ttid"] == ttid:
            ret.append(row)
    return ret

def ttids(data):
    ret = []
    for row in data:
        ret.append(row["ttid"])
    return sorted(list(set(ret)))

In [23]:
xmltree = ET.parse(data_sources[0])
xmlroot = xmltree.getroot()
timeelem = xmlroot.find(timetag)
dataTime = int(timeelem.text)


In [24]:
%%time
goods = []
for shop in xmlroot.findall(shopTag):
    shopgoods = processShop(shop, dataTime)
    for shops in shopgoods:
        goods.extend(shops)
ttid_list = ttids(goods)


TypeError: string indices must be integers

In [25]:
## resell
minSell = 0
maxBuy =  10 ** 10
result = []
for ttid in ttid_list:
    for good in selectTtid(ttid, goods):
        if good["sell"] > 0 and good["sell"] < minSell:
            minSell = good["sell"]
        if good["buy"] > 0 and good["buy"] > maxBuy:
            maxBuy = good["buy"]
    if minSell < maxBuy and minSell > 0:
        result.append([good["thingname"], minSell, maxBuy])


df = pd.DataFrame(result, columns=["name", "skupka", "prodaja"])
df

NameError: name 'ttid_list' is not defined